In [1]:
from toolbox import *
from music21 import *

In [2]:
file = converter.parse('stalker.mid')


In [3]:
file.parts[2].measure(52)
file.measure(53).parts[2]

<music21.stream.Part 0x7f390eaa82b0>

In [4]:
# for track in len(file:
for i in range(len(file.parts)):
    print(len(file.parts[i]))

77
73
51
77
77
43
58
77
73
76
77
77
77
77
77
57
77


In [5]:
for part in file.measure(2):
    print(get_list_entropy(get_pitches(part)))

0.0
0.6365141682948128
0
0
0
0
0
0
0
0
0
0
0.0
0
0
0
0


In [6]:
def get_entropy_from_measure(file,track,measure):
    E_p = get_pitches(file.measure(measure).parts[track])
    E_r = get_rhythm(file.measure(measure).parts[track])
    E_s = get_rests(file.measure(measure).parts[track])
    total_entropy = get_list_entropy(E_p) + get_list_entropy(E_r)+ get_list_entropy(E_s)
    return total_entropy

In [32]:
import random 
for i in random.sample(range(0, 16),16):
    for j in random.sample(range(1, 77),10): 
        print(i,j)
        print(get_entropy_from_measure(file,i,j),'\n')

8 19
0.0 

8 53
1.8317147343226827 

8 49
0.0 

8 60
1.4283157666600441 

8 66
1.4101760027783539 

8 73
1.8317147343226827 

8 32
0.0 

8 70
1.4101760027783539 

8 10
0.0 

8 51
1.8317147343226827 

7 57
1.8317147343226827 

7 42
0.0 

7 14
0.0 

7 35
1.8317147343226827 

7 66
1.4101760027783539 

7 75
1.8317147343226827 

7 56
1.4283157666600441 

7 74
1.4101760027783539 

7 63
1.8317147343226827 

7 49
0.0 

15 35
0.0 

15 28
0.0 

15 21
0.0 

15 54
0.6890092384766586 

15 27
0.0 

15 60
0 

15 57
0.6172417697303416 

15 1
0.0 

15 51
0.6172417697303416 

15 76
0 

6 41
0.0 

6 76
0 

6 7
0.0 

6 14
0.0 

6 19
0.0 

6 27
0.0 

6 70
0 

6 64
0 

6 54
0.6931471805599453 

6 51
0.0 

14 26
0.0 

14 32
0.0 

14 68
0.0 

14 58
0.0 

14 55
0.0 

14 28
0.0 

14 46
0.0 

14 14
0.0 

14 54
0.0 

14 64
0.0 

1 32
0.0 

1 37
0.0 

1 11
1.0549201679861442 

1 59
1.8317147343226827 

1 5
1.0114042647073516 

1 42
1.2730283365896256 

1 46
1.2730283365896256 

1 61
1.8317147343226827 

1 17
1.320

In [8]:
file.measure(1).parts[2]

<music21.stream.Part 0x7f390e920fa0>

In [34]:
from pyqubo import Binary, Constraint

def max_num_measures(file):
    return max([len(p) for p in file.parts])    

def qubo_form_from(file):
    H = 0
    for j in range(1,max_num_measures(file)):
        c = 0
        o = 0
        for i in range(len(file.parts)):
            if file.parts[i].measure(j) != None:
                c += Binary(f"x_{i}_{j}")
                o += get_entropy_from_measure(file,i,j)*Binary(f"x_{i}_{j}")
        H += Constraint((2-c)**2, f"measure_{j}") + o
    return H.compile().to_qubo()

    
# def qubo_form

In [35]:
qubo_form = qubo_form_from(file)

In [38]:
qubo_form[0]

{('x_14_44', 'x_14_44'): -3.0,
 ('x_14_55', 'x_14_55'): -3.0,
 ('x_5_36', 'x_5_36'): -1.7269716634103744,
 ('x_10_50', 'x_10_50'): -1.208240530771945,
 ('x_6_38', 'x_6_38'): -3.0,
 ('x_15_3', 'x_15_3'): -3.0,
 ('x_16_9', 'x_16_9'): -3.0,
 ('x_0_42', 'x_0_42'): -3.0,
 ('x_9_74', 'x_9_74'): -3.0,
 ('x_13_45', 'x_13_45'): -0.22741127776021886,
 ('x_10_73', 'x_10_73'): -0.9053590611739377,
 ('x_13_11', 'x_13_11'): -3.0,
 ('x_4_20', 'x_4_20'): -3.0,
 ('x_15_20', 'x_15_20'): -3.0,
 ('x_3_42', 'x_3_42'): -3.0,
 ('x_10_70', 'x_10_70'): -1.208240530771945,
 ('x_4_59', 'x_4_59'): -3.0,
 ('x_3_53', 'x_3_53'): -3.0,
 ('x_12_24', 'x_12_24'): -3.0,
 ('x_16_10', 'x_16_10'): -3.0,
 ('x_0_51', 'x_0_51'): -3.0,
 ('x_11_68', 'x_11_68'): -1.0383251960594138,
 ('x_4_16', 'x_4_16'): -3.0,
 ('x_14_60', 'x_14_60'): -3.0,
 ('x_1_72', 'x_1_72'): -1.5716842333399559,
 ('x_4_15', 'x_4_15'): -3.0,
 ('x_2_35', 'x_2_35'): -3.0,
 ('x_7_33', 'x_7_33'): -1.1682852656773173,
 ('x_12_72', 'x_12_72'): -3.0,
 ('x_15_48', '

In [69]:
from dwave.system import EmbeddingComposite, DWaveSampler
from neal import SimulatedAnnealingSampler
import pandas as pd
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(qubo_form[0],
                         num_reads = 5,
                         label='Example - Chords as QUBO')
print(sampleset.first)

Sample(sample={'x_0_1': 0, 'x_0_10': 0, 'x_0_11': 0, 'x_0_12': 0, 'x_0_13': 0, 'x_0_14': 0, 'x_0_15': 0, 'x_0_16': 0, 'x_0_17': 1, 'x_0_18': 0, 'x_0_19': 0, 'x_0_2': 0, 'x_0_20': 0, 'x_0_21': 0, 'x_0_22': 0, 'x_0_23': 0, 'x_0_24': 0, 'x_0_25': 0, 'x_0_26': 0, 'x_0_27': 0, 'x_0_28': 0, 'x_0_29': 1, 'x_0_3': 0, 'x_0_30': 0, 'x_0_31': 0, 'x_0_32': 0, 'x_0_33': 0, 'x_0_34': 0, 'x_0_35': 0, 'x_0_36': 1, 'x_0_37': 0, 'x_0_38': 0, 'x_0_39': 0, 'x_0_4': 0, 'x_0_40': 1, 'x_0_41': 0, 'x_0_42': 0, 'x_0_43': 1, 'x_0_44': 0, 'x_0_45': 0, 'x_0_46': 0, 'x_0_47': 0, 'x_0_48': 0, 'x_0_49': 0, 'x_0_5': 1, 'x_0_50': 1, 'x_0_51': 0, 'x_0_52': 1, 'x_0_53': 1, 'x_0_54': 1, 'x_0_55': 0, 'x_0_56': 0, 'x_0_57': 0, 'x_0_58': 0, 'x_0_59': 1, 'x_0_6': 1, 'x_0_60': 0, 'x_0_61': 0, 'x_0_62': 1, 'x_0_63': 0, 'x_0_64': 0, 'x_0_65': 0, 'x_0_66': 0, 'x_0_67': 0, 'x_0_68': 0, 'x_0_69': 0, 'x_0_7': 0, 'x_0_70': 0, 'x_0_71': 0, 'x_0_72': 0, 'x_0_73': 0, 'x_0_74': 0, 'x_0_75': 1, 'x_0_76': 0, 'x_0_8': 0, 'x_0_9': 0, 'x_10_

In [74]:
sampleset.to_pandas_dataframe()

,x_0_1,x_0_10,x_0_11,x_0_12,x_0_13,x_0_14,x_0_15,x_0_16,x_0_17,x_0_18,...,x_9_71,x_9_72,x_9_73,x_9_74,x_9_75,x_9_76,x_9_8,x_9_9,energy,num_occurrences
0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,1,1,0,0,0,-304.0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,-304.0,1
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,-304.0,1
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-304.0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,-304.0,1


In [83]:
sampleset.first[0]

{'x_0_1': 0,
 'x_0_10': 0,
 'x_0_11': 0,
 'x_0_12': 0,
 'x_0_13': 0,
 'x_0_14': 0,
 'x_0_15': 0,
 'x_0_16': 0,
 'x_0_17': 1,
 'x_0_18': 0,
 'x_0_19': 0,
 'x_0_2': 0,
 'x_0_20': 0,
 'x_0_21': 0,
 'x_0_22': 0,
 'x_0_23': 0,
 'x_0_24': 0,
 'x_0_25': 0,
 'x_0_26': 0,
 'x_0_27': 0,
 'x_0_28': 0,
 'x_0_29': 1,
 'x_0_3': 0,
 'x_0_30': 0,
 'x_0_31': 0,
 'x_0_32': 0,
 'x_0_33': 0,
 'x_0_34': 0,
 'x_0_35': 0,
 'x_0_36': 1,
 'x_0_37': 0,
 'x_0_38': 0,
 'x_0_39': 0,
 'x_0_4': 0,
 'x_0_40': 1,
 'x_0_41': 0,
 'x_0_42': 0,
 'x_0_43': 1,
 'x_0_44': 0,
 'x_0_45': 0,
 'x_0_46': 0,
 'x_0_47': 0,
 'x_0_48': 0,
 'x_0_49': 0,
 'x_0_5': 1,
 'x_0_50': 1,
 'x_0_51': 0,
 'x_0_52': 1,
 'x_0_53': 1,
 'x_0_54': 1,
 'x_0_55': 0,
 'x_0_56': 0,
 'x_0_57': 0,
 'x_0_58': 0,
 'x_0_59': 1,
 'x_0_6': 1,
 'x_0_60': 0,
 'x_0_61': 0,
 'x_0_62': 1,
 'x_0_63': 0,
 'x_0_64': 0,
 'x_0_65': 0,
 'x_0_66': 0,
 'x_0_67': 0,
 'x_0_68': 0,
 'x_0_69': 0,
 'x_0_7': 0,
 'x_0_70': 0,
 'x_0_71': 0,
 'x_0_72': 0,
 'x_0_73': 0,
 'x_0_74': 0,

In [96]:
sampleset.first[0].keys()[]

TypeError: 'dict_keys' object is not subscriptable

8